# Get raw block data for `blocks.json`

This file creates the raw `blocks.json` data containing information from all blocks

## Imports

In [1]:
# python3.5
import csv
import json
import os.path # os.path.isfile; 

In [2]:
# custom imports 
import util
from importlib import reload
reload(util)

<module 'util' from '/home/matteo/deep_dive/util.py'>

## Global variables and functions

In [3]:
# data up to blockheight 
current_blockheight = util.CURRENT_BLOCKHEIGHT
print(current_blockheight)

556400


### Input

#### `bitcoin_coinbases_and_headers_0-$(current_blockheight).csv`
The raw block data from bitcoin block headers and their respective coinbase transactions
address,hash,time,height,coinbase,coinbase_addresses
* `address`: One of the coinbase output addresses of this block
* `hash`: The hash of this block
* `time`: The unix timestamp of this block
* `height`: The height of this block 
* `coinbase`: The coinbase field of this block 
* `coinbase_addresses`: The number of unique coinbase output addresses

UPDATE 2019-01-28:
* `payout`: Total of BTC (in satoshis) payed out with this block (reward plus fees)
* `phash`: Hash of previous block

In [4]:
bitcoin_coinbases_and_headers_csv_file = './dataset/bitcoin_coinbases_and_headers_0-' + str(current_blockheight) + '.csv'
assert(os.path.isfile(bitcoin_coinbases_and_headers_csv_file))

## Output

#### `blocks_0-$(current_blockheight).json`
The block.json file filled with raw data form the blockchain

In [5]:
blocks_json_file = './dataset/blocks_0-' + str(current_blockheight) + '.json'
if os.path.isfile(blocks_json_file):
    print("Output file " + blocks_json_file +  " exists, will be overwritten.")

Output file ./dataset/blocks_0-556400.json exists, will be overwritten.


## Add entries to blocks file

In [6]:
blocks = dict()
blocks.clear()

with open(bitcoin_coinbases_and_headers_csv_file) as bitcoin_coinbases_and_headers_fp:
    btc_coinbases_and_headers = csv.DictReader(bitcoin_coinbases_and_headers_fp)

    for i,row in enumerate(btc_coinbases_and_headers):
        
        # line,address,bhash,time,height,coinbase,coinbase_addresses,payout
        util.add_block(blocks, 
                       height=row["height"], 
                       time=int(row["time"]), 
                       address=row["address"], 
                       cb=row["coinbase"],
                       bhash=row["hash"],
                       #phash=row["phash"],
                       payout=row["payout"])    

In [7]:
blocks['500000']

{'time': 1513622125,
 'cb': '0320a107046f0a385a632f4254432e434f4d2ffabe6d6dbdd0ee86f9a1badfd0aa1b3c9dac8d90840cf973f7b2590d6c9adde1a6e0974a010000000000000001283da9a172020000000000',
 'addresses': ['34qkc2iac6RsyxZVfyE2S5U5WcRsbg2dpK'],
 'miner': '',
 'conflicts': 0,
 'attribution': '',
 'attributions': {},
 'hash': '00000000000000000024fb37364cbf81fd49cc2d51c09c75c35433c3a1945d04',
 'phash': None,
 'payout': '1589351625'}

### Account for blocks bloopers ;) aka. strange blocks
https://bitcoin.stackexchange.com/questions/38994/will-there-be-21-million-bitcoins-eventually/38998

https://blockchain.info/de/tx/d5d27987d2a3dfc724e359870c6644b40e497bdc0589a033220fe15429d88599

In [8]:
# Check for missing blocks
for i in range(current_blockheight):
    if str(i) not in blocks.keys():
        print(i)
        #break

91722
91812
501726


In [9]:
# Apply hot fix for missing blocks:
if "91722" not in blocks.keys():
    # https://blockchain.info/rawblock/%2000000000000271a2dc26e7667f8419f2e15416dc6955e5a6c6cdf3f2574dd08e
    blocks["91722"] = { util.D_TIME:1289723848, 
                        util.D_CB:"0456720e1b00",
                        util.D_ADDRESSES:[],
                        util.D_ATTRIBUTIONS:{},
                        util.D_ATTRIBUTION:"",
                        util.D_CONFLICTS:0,
                        util.D_MINER:"",
                        util.D_HASH:"00000000000271a2dc26e7667f8419f2e15416dc6955e5a6c6cdf3f2574dd08e",
                        util.D_PHASH: "00000000000a30044feb1a9010445c5b6d4cdc3f32ca747cff2525c32976ba42",
                        util.D_PAYOUT: 5000000000 }
    
if "91812" not in blocks.keys():
    # https://blockchain.info/rawblock/00000000000af0aed4792b1acee3d966af36cf5def14935db8de83d6f9306f2f
    blocks["91812"] = { util.D_TIME:1289757588, 
                        util.D_CB:"0456720e1b00",
                        util.D_ADDRESSES:[],
                        util.D_ATTRIBUTIONS:{},
                        util.D_ATTRIBUTION:"",
                        util.D_CONFLICTS:0,
                        util.D_MINER:"",
                        util.D_HASH: "00000000000af0aed4792b1acee3d966af36cf5def14935db8de83d6f9306f2f",
                        util.D_PHASH: "000000000002afe839294d4e038b5c831bc09632fd717c0980f8f216dc2b360f",
                        util.D_PAYOUT: 5000000000}
    
if "501726" not in blocks.keys():
    # https://blockchain.info/rawblock/0000000000000000004b27f9ee7ba33d6f048f684aaeb0eea4befd80f1701126
    blocks["501726"] = { util.D_TIME:1514638520, 
                         util.D_CB:"03dea707055a478cb801b80100006ea50000", 
                         util.D_ADDRESSES:[], 
                         util.D_ATTRIBUTIONS:{},
                         util.D_ATTRIBUTION:"",
                         util.D_CONFLICTS:0,
                         util.D_MINER:"",
                         util.D_HASH: "0000000000000000004b27f9ee7ba33d6f048f684aaeb0eea4befd80f1701126",
                         util.D_PHASH: "0000000000000000002b5382b8d2d64f0b7caee90fc9951fd5d4d64b99f926bb",
                         util.D_PAYOUT: 0 }

In [10]:
with open(blocks_json_file, 'w') as outfile:
    json.dump(blocks, outfile)